In [1]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/Users/hetavpatel/Desktop/Data Science/Grad DS Work/DSCI 789 Explainable AI/XAI-Cybersec/XAI-CyberSec/tgnnexplainer/soc-redditHyperlinks-body.tsv',sep='\t')
# df=rows_count = df[df.iloc[:, 3] == 1]

In [4]:
#Transform features into seperate columns.
split_df = df['PROPERTIES'].str.split(',', expand=True)
split_df=split_df.apply(pd.to_numeric, errors='coerce').astype('float64')

In [5]:
df.columns=['i','u', 'POST_ID', 'ts',
       'sentiment', 'PROPERTIES']

In [6]:
#normalize all numerical columns
normed_df = split_df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [7]:
grouped = df.groupby(['u', 'i']).size().reset_index(name='counts')
ndf = df.merge(grouped, on=['u', 'i'], how='left')

In [8]:
ndf.columns=['i','u', 'POST_ID', 'ts',
       'sentiment', 'PROPERTIES','counts']

In [9]:
ndf = ndf[['i', 'u', 'ts', 'sentiment', 'counts']]
finaldf=pd.concat([ndf,normed_df],axis=1)

# Keep the original column names for the first three columns
new_col_names = finaldf.columns[:3].tolist()

# Generate new names for the rest of the columns
new_col_names += ['f' + str(i+1) for i in range(finaldf.shape[1]-3)]

# Create a dictionary of old column names to new column names
col_name_dict = dict(zip(finaldf.columns, new_col_names))

# Rename columns
xdf = finaldf.rename(columns=col_name_dict)

In [10]:
#forgot to normalize these columns
xdf.iloc[:, 3] = (xdf.iloc[:, 3] - xdf.iloc[:, 3].min()) / (xdf.iloc[:, 3].max() - xdf.iloc[:, 3].min())
xdf.iloc[:, 4] = (xdf.iloc[:, 4] - xdf.iloc[:, 4].min()) / (xdf.iloc[:, 4].max() - xdf.iloc[:, 4].min())

/var/folders/xr/0vdqp3k12gn7yn75q4893hsm0000gn/T/ipykernel_83232/3159772687.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         1.0
1         0.0
2         1.0
3         1.0
4         1.0
         ... 
286556    1.0
286557    1.0
286558    1.0
286559    1.0
286560    1.0
Name: f1, Length: 286561, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  xdf.iloc[:, 3] = (xdf.iloc[:, 3] - xdf.iloc[:, 3].min()) / (xdf.iloc[:, 3].max() - xdf.iloc[:, 3].min())
/var/folders/xr/0vdqp3k12gn7yn75q4893hsm0000gn/T/ipykernel_83232/3159772687.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         0.069470
1         0.000000
2         0.000000
3         0.023766
4         0.020110
            ...   
286556    0.000000
286557    0.000000
286558    0.016453
286559    0.000000
286560    0.000000
N

In [11]:
# Convert the column to datetime format
xdf['ts'] = pd.to_datetime(xdf['ts'])

# Convert datetime column to Unix timestamp
xdf['ts'] = (xdf['ts'].astype(int) / 1e9).astype(int)

In [12]:
# Combine the values from both columns
combined = pd.concat([xdf['u'], xdf['i']])

# Get the unique values and factorize
labels, unique = pd.factorize(combined.unique())

# Create a dictionary mapping string to integer ID
mapping = dict(zip(unique, labels))

# Use the dictionary to map the string values to integer ID in the original DataFrame
xdf['u'] = xdf['u'].map(mapping)
xdf['i'] = xdf['i'].map(mapping)

In [13]:
print(xdf.iloc[:, 0].max())
print(xdf.iloc[:, 0].nunique())

35775
27863


In [14]:
#add label column, not relevant for unipartite graph.
xdf['label']=0

In [15]:
xdf.head(5)

,i,u,ts,f1,f2,f3,f4,f5,f6,f7,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,label
0,55,0,1388507998,1.0,0.069470,0.007186,0.006333,0.754070,0.020411,0.088093,...,0.110405,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
1,5599,1,1388513917,0.0,0.000000,0.001262,0.001246,0.739638,0.023245,0.049930,...,0.213450,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
2,4502,2,1388588075,1.0,0.000000,0.000874,0.000916,0.750365,0.027626,0.083402,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
3,44,3,1388511475,1.0,0.023766,0.026099,0.022889,0.770335,0.002060,0.058413,...,0.066703,0.0,0.0,0.0,0.0,0.0,0.0,0.203125,0.017857,0
4,3835,4,1388544673,1.0,0.020110,0.016169,0.014573,0.775902,0.008189,0.025138,...,0.080044,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.028571,0


In [16]:
xdf.shape

(286561, 92)

In [17]:
xdf.to_csv('reddit_hyperlinks.csv')

In [ ]:
unique_count_first_column = df.iloc[:, 0].nunique()
unique_count_second_column = df.iloc[:, 1].nunique()
unique_count_third_column = df.iloc[:, 3].nunique()

print(f"Number of unique values in the first column: {unique_count_first_column}")
print(f"Number of unique values in the second column: {unique_count_second_column}")
print(f"Number of unique values in the third column: {unique_count_third_column}")


nonzero_count_first_column = np.count_nonzero(df.iloc[:, 0])
nonzero_count_second_column = np.count_nonzero(df.iloc[:, 1])
nonzero_count_third_column = np.count_nonzero(df.iloc[:, 2])

print(f"Number of non-zero values in the first column: {nonzero_count_first_column}")
print(f"Number of non-zero values in the second column: {nonzero_count_second_column}")
print(f"Number of non-zero values in the third column: {nonzero_count_third_column}")

Number of unique values in the first column: 27863
Number of unique values in the second column: 20606
Number of unique values in the third column: 248453
Number of non-zero values in the first column: 286561
Number of non-zero values in the second column: 286561
Number of non-zero values in the third column: 286561
